In [ ]:
"""
Purpose: To check on the status of the soma finder


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [2]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-20 00:52:26,003 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-20 00:52:26,005 - settings - Setting database.user to celiib
INFO - 2020-11-20 00:52:26,006 - settings - Setting database.password to newceliipass
INFO - 2020-11-20 00:52:26,009 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-20 00:52:26,009 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 00:52:26,022 - connection - Connect

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Chccking the current progress of Soma Extraction

In [ ]:
dj.config["display.limit"] = 50

In [ ]:
minnie.BaylorSegmentCentroid() & "n_vertices>0"

In [9]:
minnie.BaylorSegmentCentroid()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,soma_vertices array of vertices,soma_faces array of faces,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691135617760911,0,0.25,1,269148,139228,24891,12205,22777,=BLOB=,=BLOB=,1,0.85,1.4,2.387,6094628.542,355.2345


In [ ]:
error_key = (schema.jobs & "table_name='__baylor_segment_centroid'" & "status='error'" &
 "error_message='TypeError: only integer scalar arrays can be converted to a scalar index'").fetch("key")
# "error_message!='ValueError: zero-size array to reduction operation maximum which has no identity'")


In [ ]:
len(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()

# checking the soma extraction

In [18]:
curr_seg_id = 864691135855647534
pulled_mesh = du.fetch_segment_id_mesh(curr_seg_id)

INFO - 2020-11-20 01:01:45,383 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 01:01:45,594 - settings - Setting enable_python_native_blobs to True


In [7]:
undec_mesh = du.fetch_undecimated_segment_id_mesh(curr_seg_id)

INFO - 2020-11-20 00:56:29,483 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 00:56:29,642 - settings - Setting enable_python_native_blobs to True


In [19]:
du = reload(du)
somas = du.get_soma_mesh_list(curr_seg_id)
somas

INFO - 2020-11-20 01:01:59,424 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-20 01:01:59,424 - settings - Setting database.user to celiib
INFO - 2020-11-20 01:01:59,425 - settings - Setting database.password to newceliipass
INFO - 2020-11-20 01:01:59,428 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 01:01:59,620 - settings - Setting enable_python_native_blobs to True


[[<trimesh.Trimesh(vertices.shape=(38794, 3), faces.shape=(73257, 3))>],
 [203.7982],
 [0.611]]

In [20]:
nviz.plot_objects(meshes=somas[0])

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [21]:
import neuron_visualizations as nviz
nviz.plot_objects(pulled_mesh)

In [ ]:
tu.largest_hole_length(soma_info[0][0])

In [ ]:
sm = reload(sm)
soma_info = sm.extract_soma_center(curr_seg_id,
                      current_mesh_verts=pulled_mesh.vertices,
                      current_mesh_faces=pulled_mesh.faces)

In [ ]:
soma_info

In [ ]:
nviz.plot_objects(meshes=soma_info[0])